In [75]:
import os, zipfile
from os import path
import requests
from requests.auth import HTTPBasicAuth
import subprocess
import datetime as dt
from datetime import datetime, date, timedelta
from pymongo import MongoClient

In [ ]:
# this doesn't work and I don't know if I need it
# subprocess.Popen(['//usr//bin//mongod', '--dbpath',  '~//Metis//NBM_Engineering_Student//my_project//datadb'])

In [ ]:
# from urllib.parse import quote_plus

# uri = "mongodb://%s:%s@%s" % (
#     quote_plus(user), quote_plus(password), host)
# client = MongoClient(uri)

In [2]:
# create a new client instance
mongo_client = MongoClient()

In [3]:
mongo_client.test_database

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test_database')

In [99]:
# account details
USERNAME = 'melissa'
PASSWORD = 'WattTime4Metis'
EMAIL = 'melissa@doloresparkpiano.com'
ORG = 'StudentsInc'

# request details
BA = 'CAISO_NORTH'  # identify grid region

# starttime and endtime are optional, if ommited will return the latest value
START = '2021-09-08T19:00:00-0000'  # UTC offset of 0 (PDT is -7, PST -8)
END = '2021-09-08T19:45:00-0000'


def register(username, password, email, org):
    url = 'https://api2.watttime.org/register'
    params = {'username': username,
              'password': password,
              'email': email,
              'org': org}
    rsp = requests.post(url, json=params)
    print(rsp.text)


def login(username, password):
    url = 'https://api2.watttime.org/login'
    try:
        rsp = requests.get(url, auth=HTTPBasicAuth(username, password))
    except BaseException as e:
        print('There was an error making your login request: {}'.format(e))
        return None

    try:
        token = rsp.json()['token']
    except BaseException:
        print('There was an error logging in. The message returned from the '
              'api is {}'.format(rsp.text))
        return None

    return token


def create_token(USERNAME, PASSWORD):
    token = login(USERNAME, PASSWORD)
    if not token:
        print('You will need to fix your login credentials (username and password '
              'at the start of this file) before you can query other endpoints. '
              'Make sure that you have registered at least once by uncommenting '
              'the register(username, password, email, org) line near the bottom '
              'of this file.')
        exit()
    return token


def data(token, ba, starttime, endtime):
    url = 'https://api2.watttime.org/data'
    headers = {'Authorization': 'Bearer {}'.format(token)}
    params = {'ba': ba, 'starttime': starttime, 'endtime': endtime}

    rsp = requests.get(url, headers=headers, params=params)
    # print(rsp.text)  # uncomment to see raw response
    return rsp.json()


def index(token, ba):
    url = 'https://api2.watttime.org/index'
    headers = {'Authorization': 'Bearer {}'.format(token)}
    params = {'ba': ba}

    rsp = requests.get(url, headers=headers, params=params)
    # print(rsp.text)  # uncomment to see raw response
    return rsp.json()


def forecast(token, ba, starttime=None, endtime=None):
    url = 'https://api2.watttime.org/forecast'
    headers = {'Authorization': 'Bearer {}'.format(token)}
    params = {'ba': ba}
    if starttime:
        params.update({'starttime': starttime, 'endtime': endtime})

    rsp = requests.get(url, headers=headers, params=params)
    # print(rsp.text)  # uncomment to see raw response
    return rsp.json()


def historical(token, ba):
    url = 'https://api2.watttime.org/historical'
    headers = {'Authorization': 'Bearer {}'.format(token)}
    params = {'ba': ba}
    rsp = requests.get(url, headers=headers, params=params)
    cur_dir = path.dirname(path.realpath('my_project'))
    file_dir = path.join(cur_dir, '{}_historical'.format(ba))
    file_name = '{}_historical.zip'.format(ba)
    file_path = path.join(file_dir, file_name)
    with open(file_path, 'wb') as fp:
        fp.write(rsp.content)

    print('Wrote historical data for {} to {}'.format(ba, file_path))
    return file_dir, file_path

In [82]:
# Only register once!!
# register(USERNAME, PASSWORD, EMAIL, ORG)

In [188]:
mongo_client.watt_time.list_collection_names()

['CAISO_NORTH']

In [189]:
# use this for collection reset/rerun experiments
# mongo_client.watt_time[BA].drop()

In [186]:
db_names = mongo_client.list_database_names()
print(db_names)
coll_names = mongo_client.watt_time.list_collection_names()
print(coll_names)

['admin', 'books', 'config', 'events', 'local', 'outings', 'watt_time']
['CAISO_NORTH']


In [187]:
BA not in coll_names

False

In [157]:
if BA not in db_names:
    token = create_token(USERNAME, PASSWORD)
    file_dir, file_path = historical(token, BA)
    
    # extract csv files from historical zip file
    zip_ref = zipfile.ZipFile(file_path)
    zip_ref.extractall(file_dir)
    zip_ref.close()
    os.remove(file_path)
    
    # create a new collection and insert data
    db = mongo_client.watt_time
    new_coll = db[BA]
    
    # directory of files
    dir_files = file_dir
    # create list of all files
    _, _, file_names = next(os.walk(file_dir))
    files = [os.path.join(file_dir, fn) for fn in file_names]
    # mongotool address
    mongotool = r'/usr/bin/mongoimport'
    # import all files to mongodb
    for file in files:
        print(file)
        commands = [mongotool, '--db', 'watt_time',
                    '--collection', BA,
                    '--file', file,
                    '--type', 'csv',
                    '--headerline']
        commands = ' '.join(commands)
        subprocess.Popen(commands, shell=True)
    
    # check most recent date
    cursor = new_coll.find().sort('timestamp', -1).limit(1)
    latest_date = list(cursor)[0]['timestamp']
    print(latest_date)

    # the historical zip timestamp & MOER are renamed in subsequent API calls
    new_coll.update_many({}, {'$rename': {"timestamp": "point_time", "MOER": "value"}}, False, array_filters=None)
    
    # same check plus check rename
    cursor = new_coll.find().sort('point_time', -1).limit(1)
    latest_date = list(cursor)[0]['point_time']
    print(latest_date)
    

    

Wrote historical data for CAISO_NORTH to /home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_historical.zip
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2020-08_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2018-07_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2018-10_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2020-10_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2021-04_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2018-06_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2021-08_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2018-04_MOER.cs

2021-09-12T17:03:31.193-0700	connected to: localhost
2021-09-12T17:03:31.201-0700	connected to: localhost
2021-09-12T17:03:31.204-0700	connected to: localhost
2021-09-12T17:03:31.212-0700	connected to: localhost
2021-09-12T17:03:31.227-0700	connected to: localhost
2021-09-12T17:03:31.240-0700	connected to: localhost
2021-09-12T17:03:31.243-0700	connected to: localhost
2021-09-12T17:03:31.256-0700	connected to: localhost
2021-09-12T17:03:31.293-0700	connected to: localhost
2021-09-12T17:03:31.300-0700	imported 8928 documents
2021-09-12T17:03:31.300-0700	connected to: localhost
2021-09-12T17:03:31.301-0700	connected to: localhost
2021-09-12T17:03:31.319-0700	imported 8928 documents
2021-09-12T17:03:31.332-0700	imported 8928 documents
2021-09-12T17:03:31.339-0700	connected to: localhost
2021-09-12T17:03:31.355-0700	imported 8640 documents
2021-09-12T17:03:31.361-0700	connected to: localhost
2021-09-12T17:03:31.366-0700	connected to: localhost
2021-09-12T17:03:31.367-0700	imported 8928 doc

/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2020-01_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2021-01_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2021-02_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2019-07_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2018-09_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2021-06_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2021-07_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2019-06_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2019-02_MOER.csv
/home/melissa/Metis/NBM_Engineering_S

2021-09-12T17:03:31.426-0700	imported 8640 documents
2021-09-12T17:03:31.433-0700	imported 8640 documents
2021-09-12T17:03:31.456-0700	connected to: localhost
2021-09-12T17:03:31.459-0700	connected to: localhost
2021-09-12T17:03:31.468-0700	imported 8640 documents
2021-09-12T17:03:31.469-0700	connected to: localhost
2021-09-12T17:03:31.497-0700	imported 8928 documents
2021-09-12T17:03:31.507-0700	imported 8640 documents
2021-09-12T17:03:31.508-0700	connected to: localhost
2021-09-12T17:03:31.523-0700	connected to: localhost
2021-09-12T17:03:31.527-0700	imported 8928 documents
2021-09-12T17:03:31.531-0700	imported 8928 documents
2021-09-12T17:03:31.535-0700	connected to: localhost
2021-09-12T17:03:31.535-0700	imported 8640 documents
2021-09-12T17:03:31.543-0700	imported 8928 documents
2021-09-12T17:03:31.546-0700	imported 8928 documents
2021-09-12T17:03:31.565-0700	connected to: localhost
2021-09-12T17:03:31.569-0700	connected to: localhost
2021-09-12T17:03:31.569-0700	connected to: loc

/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2019-01_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2020-09_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2019-10_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2021-05_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2018-03_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2020-05_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2020-12_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2020-06_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2018-01_MOER.csv
/home/melissa/Metis/NBM_Engineering_S

2021-09-12T17:03:31.644-0700	imported 8640 documents
2021-09-12T17:03:31.652-0700	connected to: localhost
2021-09-12T17:03:31.665-0700	imported 8064 documents
2021-09-12T17:03:31.665-0700	imported 8640 documents
2021-09-12T17:03:31.681-0700	imported 8928 documents
2021-09-12T17:03:31.685-0700	connected to: localhost
2021-09-12T17:03:31.705-0700	imported 8928 documents
2021-09-12T17:03:31.708-0700	connected to: localhost
2021-09-12T17:03:31.721-0700	connected to: localhost
2021-09-12T17:03:31.728-0700	imported 8640 documents
2021-09-12T17:03:31.743-0700	connected to: localhost
2021-09-12T17:03:31.756-0700	imported 8928 documents
2021-09-12T17:03:31.759-0700	imported 8928 documents
2021-09-12T17:03:31.762-0700	imported 8928 documents
2021-09-12T17:03:31.765-0700	imported 8640 documents
2021-09-12T17:03:31.798-0700	connected to: localhost
2021-09-12T17:03:31.804-0700	imported 8928 documents
2021-09-12T17:03:31.804-0700	connected to: localhost
2021-09-12T17:03:31.805-0700	connected to: loc

/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2019-03_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2018-08_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2018-05_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2018-02_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2019-09_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2018-11_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2020-03_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2019-08_MOER.csv
/home/melissa/Metis/NBM_Engineering_Student/my_project/CAISO_NORTH_historical/CAISO_NORTH_2020-04_MOER.csv


2021-09-12T17:03:31.850-0700	imported 8928 documents
2021-09-12T17:03:31.870-0700	connected to: localhost
2021-09-12T17:03:31.873-0700	connected to: localhost
2021-09-12T17:03:31.880-0700	imported 8928 documents
2021-09-12T17:03:31.886-0700	imported 8928 documents
2021-09-12T17:03:31.902-0700	imported 8928 documents
2021-09-12T17:03:31.907-0700	connected to: localhost
2021-09-12T17:03:31.922-0700	imported 8352 documents
2021-09-12T17:03:31.936-0700	connected to: localhost
2021-09-12T17:03:31.937-0700	connected to: localhost
2021-09-12T17:03:31.957-0700	imported 8640 documents
2021-09-12T17:03:31.957-0700	connected to: localhost
2021-09-12T17:03:31.978-0700	imported 8928 documents
2021-09-12T17:03:31.985-0700	imported 8928 documents
2021-09-12T17:03:31.986-0700	connected to: localhost
2021-09-12T17:03:32.019-0700	imported 8928 documents
2021-09-12T17:03:32.043-0700	imported 8064 documents
2021-09-12T17:03:32.052-0700	imported 8640 documents
2021-09-12T17:03:32.066-0700	imported 8928 doc

In [158]:
new_coll.count_documents({})

381609

In [167]:
# cursor = new_coll.find().sort('timestamp', -1).limit(1)
# latest_date = list(cursor)[0]['timestamp']
# latest_date

'2021-08-18T00:40:00+00:00'

In [178]:
# this needs to be done on the data that was unzipped in the if statement
# new_coll.update_many({}, {'$rename': {"timestamp": "point_time", "MOER": "value"}}, False, array_filters=None)

In [179]:
# cursor = new_coll.find().sort('point_time', -1).limit(1)
# latest_date = list(cursor)[0]['point_time']
# latest_date

'2021-08-18T00:40:00+00:00'

In [190]:
new_coll.find_one()

{'_id': ObjectId('6142d9ae6e211238d3e1e78b'),
 'MOER version': 3.0,
 'frequency': 'None',
 'point_time': '2018-10-01T00:00:00+00:00',
 'value': 964.0}

In [181]:
start_time = latest_date
thirty_days_after = latest_date
time_now = datetime.utcnow().astimezone().replace(tzinfo=dt.timezone.utc, microsecond=0).isoformat()
print(time_now)

if start_time != time_now:
    while thirty_days_after < time_now:
        thirty_days_after = (datetime.fromisoformat(latest_date) + timedelta(days=30)).isoformat()
        print(thirty_days_after)
        if thirty_days_after > time_now:
            end_time = time_now
        else:
            end_time = thirty_days_after
            
        print(end_time)
        # run /data with start/stop times to update db
        update_db = data(token, BA, start_time, end_time)
        print(len(update_db))
        
        # point_time timestamp is slightly different format from /historical timestamp
        temp_df = pd.DataFrame(update_db)
        temp_df['point_time'] = [parser.parse(x).replace(tzinfo=dt.timezone.utc, microsecond=0).isoformat() for x in temp_df['point_time']]
        
        # insert next 30 (or less) days of data into BA collection
        new_coll.insert_many(temp_df.to_dict('records'))
        

2021-09-13T00:19:14+00:00
2021-09-17T00:40:00+00:00
2021-09-13T00:19:14+00:00
7484


In [191]:
new_coll.count_documents({})

390029

In [192]:
# find latest date in collection
cursor = new_coll.find().sort('point_time', -1).limit(10)
latest_date = list(cursor)[0]['point_time']
latest_date

'2021-09-16T06:05:00+00:00'

In [199]:
db.list_collection_names()

['CAISO_NORTH_forecast',
 'CAISO_NORTH_weeklyline',
 'CAISO_NORTH_heatmap',
 'CAISO_NORTH',
 'CAISO_NORTH_dailyline']

In [217]:
db['CAISO_NORTH_forecast'].count_documents({})

181152

In [216]:
db['CAISO_NORTH'].count_documents({})

390327

In [218]:
pd.DataFrame(list(db['CAISO_NORTH_weeklyline'].find({})))

,_id,index,point_time,forecast,moer_today,moer_1week,moer_2week,moer_3week,point_date,time,weekday
0,6144396877427e0f95ae91ea,0,2021-09-07 00:00:00,928.213628,1022.0,883.0,44.0,869.0,2021-09-07,00:00:00,2
1,6144396877427e0f95ae91eb,1,2021-09-07 00:05:00,944.270083,932.0,869.0,866.0,864.0,2021-09-07,00:05:00,2
2,6144396877427e0f95ae91ec,2,2021-09-07 00:10:00,934.695229,917.0,869.0,865.0,861.0,2021-09-07,00:10:00,2
3,6144396877427e0f95ae91ed,3,2021-09-07 00:15:00,944.270083,914.0,869.0,869.0,863.0,2021-09-07,00:15:00,2
4,6144396877427e0f95ae91ee,4,2021-09-07 00:20:00,944.270083,912.0,870.0,870.0,863.0,2021-09-07,00:20:00,2
...,...,...,...,...,...,...,...,...,...,...,...
2011,6144396877427e0f95ae99c5,2011,2021-09-13 23:35:00,989.202248,NaN,1026.0,890.0,453.0,2021-09-13,23:35:00,1
2012,6144396877427e0f95ae99c6,2012,2021-09-13 23:40:00,989.202248,NaN,1025.0,888.0,43.0,2021-09-13,23:40:00,1
2013,6144396877427e0f95ae99c7,2013,2021-09-13 23:45:00,990.115426,NaN,1024.0,886.0,43.0,2021-09-13,23:45:00,1
2014,6144396877427e0f95ae99c8,2014,2021-09-13 23:50:00,990.177095,NaN,1024.0,885.0,43.0,2021-09-13,23:50:00,1
